In [75]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheusmbr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [76]:
year = 2013
df = pd.read_csv(f'./Raw/{year}.csv', sep=';', encoding ='latin')
df_processed = df[['AN_BASE', 'NM_SUBTIPO_PRODUCAO', 'NM_PROJETO', 'DS_RESUMO', 'DS_PALAVRA_CHAVE', 'DS_ABSTRACT', 'DS_KEYWORD', 'NM_AREA_AVALIACAO']].copy()

In [77]:
df_processed.head()

,AN_BASE,NM_SUBTIPO_PRODUCAO,NM_PROJETO,DS_RESUMO,DS_PALAVRA_CHAVE,DS_ABSTRACT,DS_KEYWORD,NM_AREA_AVALIACAO
0,2013,DISSERTAÇÃO,NaN,A POSSIBILIDADE DA OCORRÊNCIA DE SISMOS NO BRA...,"ANÁLISE SÍSMICA, CONCRETO ARMADO, DETALHAMENTO...",NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I
1,2013,DISSERTAÇÃO,NaN,AS OBRAS DE PONTES ESTAIADAS ENVOLVEM UMA TECN...,PONTE ESTAIADA. PONTE DO SABER. ANÁLISE ESTRUT...,NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I
2,2013,TESE,HIPERPLASIA DE CÉLULAS NEUROENDRÓCRINAS EM LAC...,A HIPERPLASIA DE CÉLULAS NEUROENDÓCRINAS DO LA...,1. DOENÇAS PULMONARES INTERSTICIAIS  RADIOGRA...,NEUROENDOCRINE CELL HYPERPLASIA OF INFANCY (NE...,NÃO INFORMADO,MEDICINA II
3,2013,TESE,AVALIAÇÃO ATRAVÉS DAS TÉCNICAS AVANÇADAS DE RM...,ESTUDOS RECENTES TEM DEMONSTRADO ALTERAÇÕES DA...,1. NEURALGIA DO TRIGÊMEO - FISIOPATOLOGIA. 2. ...,RECENT STUDIES HAVE DETECTED LOCAL CHANGES OF ...,NÃO INFORMADO,MEDICINA II
4,2013,DISSERTAÇÃO,NaN,DENTRE AS TÉCNICAS MAIS UTILIZADAS NA DETERMIN...,"PLATAFORMAS FIXAS, ESTRUTURAS OFFSHORE, ESTRUT...",NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I


In [78]:
df_processed.columns = df_processed.columns.str.slice(3).str.lower()
df_processed.rename(columns={
    'base': 'ano'
}, inplace=True)

for item in list(df_processed.iloc[:, 1:].columns):
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lower()

In [79]:
df_processed.isna().sum()

ano                     0
subtipo_producao        0
projeto             22280
resumo                  0
palavra_chave           0
abstract                7
keyword                 0
area_avaliacao          0
dtype: int64

In [80]:
df_processed.dropna(subset=['abstract'], inplace=True)

df_processed.fillna({ 'projeto': 'não informado' }, inplace=True)

# df_processed.drop(index=df_processed[
#     df_processed.resumo.str.contains("não informado") | df_processed.palavra_chave.str.contains("não informado") | 
#     df_processed.abstract.str.contains("não informado") | df_processed.keyword.str.contains("não informado") | 
#     df_processed.grande_area_conhecimento.str.contains("não informado")
# ].index, inplace=True)
# df_processed.reset_index(drop=True, inplace=True)

In [81]:
idx = df_processed[(df_processed.palavra_chave == df_processed.keyword) | (df_processed.resumo == df_processed.abstract)].index
df_processed.drop(index=idx, inplace=True)

In [82]:
for item in ['resumo', 'abstract']:
    df_processed[f'{item}'].replace(to_replace ='[0-9]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[,.:;!?]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[/<>()|\+\-\$%&#@\'\"]+', value = '', regex = True, inplace=True)

In [83]:
for item in ['palavra_chave', 'keyword']:
    df_processed[f'{item}'].replace(to_replace ='([0-9][. ])', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='([;])+', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='([.])+', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='(  )+', value = ' ', regex = True, inplace=True)
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lstrip(', ')

In [84]:
df_processed.head()

,ano,subtipo_producao,projeto,resumo,palavra_chave,abstract,keyword,area_avaliacao
0,2013,dissertação,não informado,a possibilidade da ocorrência de sismos no bra...,"análise sísmica, concreto armado, detalhamento...",não informado,não informado,engenharias i
1,2013,dissertação,não informado,as obras de pontes estaiadas envolvem uma tecn...,"ponte estaiada, ponte do saber, análise estrut...",não informado,não informado,engenharias i
2,2013,tese,hiperplasia de células neuroendrócrinas em lac...,a hiperplasia de células neuroendócrinas do la...,doenças pulmonares intersticiais  radiografia...,neuroendocrine cell hyperplasia of infancy neh...,não informado,medicina ii
3,2013,tese,avaliação através das técnicas avançadas de rm...,estudos recentes tem demonstrado alterações da...,"neuralgia do trigêmeo - fisiopatologia, , neur...",recent studies have detected local changes of ...,não informado,medicina ii
4,2013,dissertação,não informado,dentre as técnicas mais utilizadas na determin...,"plataformas fixas, estruturas offshore, estrut...",não informado,não informado,engenharias i


## Testes

In [140]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheusmbr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [141]:
test = df_processed.resumo.copy()

In [142]:
test = test.head(50)

In [143]:
def pre_processamento(text):
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in text if item not in stop_words]
    return " ".join(new_text)

In [144]:
test = test.apply(word_tokenize).apply(pre_processamento)

In [145]:
test.head()

0    possibilidade ocorrência sismos brasil possam ...
1    obras pontes estaiadas envolvem tecnologia dif...
2    hiperplasia células neuroendócrinas lactente h...
3    estudos recentes demonstrado alterações substâ...
4    dentre técnicas utilizadas determinação capaci...
Name: resumo, dtype: object

In [146]:
from nltk.stem.snowball import SnowballStemmer

def stemming_bag(words, stemmer=SnowballStemmer("portuguese")):
    return [stemmer.stem(w) for w in words]

test = test.apply(word_tokenize).apply(stemming_bag)

In [147]:
test

0     [possibil, ocorrent, sism, brasil, poss, compr...
1     [obras, pont, estai, envolv, tecnolog, diferen...
2     [hiperplas, célul, neuroendócrin, lactent, hcn...
3     [estud, recent, demonstr, alter, substânc, bra...
4     [dentr, técnic, utiliz, determin, capac, resis...
5     [ness, trabalh, objetiv, test, performanc, map...
6     [qualidad, vid, enquant, orient, polít, públic...
8     [síndrom, sjögren, ss, doenc, autoimun, comum,...
9     [proteçã, trabalh, tratas, construçã, histór, ...
10    [leishmanios, visceral, human, lvh, conhec, ca...
13    [present, dissert, traz, result, obtid, atravé...
14    [objet, dest, estud, analis, retrospect, tomog...
15    [trabalh, apresent, estud, compar, alguns, pon...
16    [tanqu, armazen, equip, industri, encontr, div...
17    [dissert, inser, linh, pesquis, literatur, mi...
18    [crescent, foment, tecnolog, fic, abertur, crí...
19    [dissert, pretend, desenvolv, estud, compar, p...
20    [pesquis, trat, relaçã, alguns, acontec, h